# Setup

In [1]:
# Install the 'swarm' library from GitHub and the 'requests' library for HTTP requests
!pip install git+https://github.com/openai/swarm.git
!pip install requests

  Cloning https://github.com/openai/swarm.git to c:\users\kondr\appdata\local\temp\pip-req-build-4rx4mcgd
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached cfgv-3.4.0-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached nodeenv-1.9.1-py2.py3-none-any.whl.metadata (21 kB)
Using cached cfgv-3.4.0-py2.py3-none-any.whl (7.2 kB)
Using cached nodeenv-1.9.1-py2.py3-none-any.whl (22 kB)
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 4.3/4.3 MB 32.1 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=26247 sh

  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git 'C:\Users\kondr\AppData\Local\Temp\pip-req-build-4rx4mcgd'


In [2]:
from dotenv import load_dotenv
import os

In [3]:
# # Set OpenAI key in the environment


# api_key = userdata.get('OPENAI_API_KEY')
# os.environ['OPENAI_API_KEY'] = api_key

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
if api_key:
    print(f"OpenAI API Key exists and begins {api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [4]:
from swarm import Swarm, Agent
import requests
from IPython.display import Markdown
import json
from bs4 import BeautifulSoup

## Motivator

In [5]:
def get_motivational_quotes():
  return requests.get('https://stoic.tekloon.net/stoic-quote').json()

In [6]:
motivator_agent = Agent(
    name="Motivator Agent",
    instructions="Motivate student with stoic motivational quotes, present the quotes and relate it to student's experience.",
    functions=[get_motivational_quotes],
)

In [7]:
def transfer_to_motivator():
    return motivator_agent

## Counselor

In [8]:
def counselor(context_variables):
    # Retrieve student's academic information
    students_academic = context_variables.get("academic_info", None)

    # Return detailed, friendly, and approachable instructions
    return f"""
    You are a high school counselor specializing in helping students improve their academic performance
    and achieve admission to their dream universities.

    provide personalized advice and guidance tailored to their goals.

    Your approach should be:
    - Friendly and supportive, like a trusted advisor and mentor.
    - Focused on giving concise, actionable recommendations.
    - Avoid overwhelming the student with a long list of tasks; instead, make it short and clear.

    Make the conversation feel approachable and engaging, ensuring the student feels encouraged and motivated.
    """

In [9]:
counselor_agent = Agent(
    name="Counselor Agent",
    instructions=counselor,
)

In [10]:
def transfer_to_counselor():
    return counselor_agent

## Crisis handler

In [11]:
def crisis_handler():
    # Return clear, friendly, and supportive instructions
    return """
    You are a high school counselor trained in managing crisis situations.
    Your primary goal is to provide reassurance and support to the student while facilitating a connection to a human counselor.

    Here's what you should do:
    - Calmly and empathetically calm the student.
    - Reassure the student that help is on the way, and a human counselor will be with them shortly.
    - Maintain a warm, non-judgmental, and supportive tone throughout the interaction.

    Remember, your role is to provide comfort and instill a sense of safety until the human counselor arrives.
    """

In [12]:
crisis_handler_agent = Agent(
    name="Crisis Handler Agent",
    instructions=crisis_handler,
)

In [13]:
def transfer_to_crisis_handler():
    return crisis_handler

## Therapist agent

In [14]:
def therapist(context_variables):
  students_info = context_variables.get("student_data", None)
  students_mood_history = context_variables.get("mood_history", None)
  return f"""
    You are a therapist specializing in supporting high school students. Current Student: {students_info}
    Your primary tasks is to consider their past mood pattern {students_mood_history} and engage in conversations to understand their current mental state.
    If the student is feeling down or unmotivated, call the motivator agent.
    If the student is focusing on their academic goals or their future education, call the counselor agent.
    If the user is feeling bad and you feel a human attention is required, call the crisis handler agent.
    Your approach should be:
    Be friendly, approachable, and encouraging. Avoid giving a long list and keep the conversation natural.
    Call one agent at a time depending on the student's mood.
    """

In [15]:
# Main Agent
therapist_agent = Agent(
    name="Therapist Agent",
    instructions=therapist,
    functions = [transfer_to_motivator, transfer_to_counselor, transfer_to_crisis_handler]
)

In [16]:
def transfer_back_to_therapist():
    return therapist_agent

In [17]:
motivator_agent.functions.append(transfer_back_to_therapist)
counselor_agent.functions.append(transfer_back_to_therapist)
crisis_handler_agent.functions.append(transfer_back_to_therapist)

## Conversation

In [18]:
def pretty_print_messages(messages) -> None:
    # Iterate over the list of messages
    for message in messages:
        # Skip messages that are not from the assistant
        if message["role"] != "assistant":
            continue

        # Print the sender's name in blue
        print(f"\033[94m{message['sender']}\033[0m:", end=" ")

        # If the message contains content, display it as Markdown
        if message["content"]:
            display(Markdown((message["content"])))

        # Extract tool calls, if any, from the message
        tool_calls = message.get("tool_calls") or []

        # Add a blank line if there are multiple tool calls
        if len(tool_calls) > 1:
            print()

        # Iterate through each tool call and print its details
        for tool_call in tool_calls:
            f = tool_call["function"]

            # Extract function name and arguments
            name, args = f["name"], f["arguments"]

            # Format arguments for display, replacing ":" with "=" for better readability
            arg_str = json.dumps(json.loads(args)).replace(":", "=")

            # Print the tool call in purple
            print(f"\033[95m{name}\033[0m({arg_str[1:-1]})")


In [19]:
context_variables = {
    "student_data": """Here is the internal data of the student:
        1. NAME: Maximus Decimus Meridius
        2. CLASS: 10-A
        3. NOTES: problem with family
        """,
    "mood_history": ['happy', 'happy', 'neutral', 'sad', 'depressed', 'neutral', 'happy', 'sad'],
    "academic_info": """
        1. CLUB: Rugby
        2. PROFESSIONAL GOAL: Biologist
        3. VOLUNTEER: None
        4. GPA: 3.1
    """
}

In [20]:
messages = []

client = Swarm()
while True:
  user_input = input("\033[90mUser\033[0m: ")

  messages.append({"role": "user", "content": user_input})

  if user_input.lower() == 'bye':
    break

  response = client.run(
      agent=therapist_agent,
      messages=messages,
      context_variables=context_variables or {},
  )

  pretty_print_messages(response.messages)

  messages.extend(response.messages)
  agent = response.agent


Therapist Agent: 

Hello Maximus! How are you doing today? I'm here to chat and see how you're feeling. Is there anything on your mind that you'd like to talk about?

Therapist Agent: 

It's okay if you don't know where to start. Remember, it's normal to feel a range of things, especially given everything you handle as a student. 

Are there any specific areas you're finding challenging right now, like school, family, or anything else? I'm here to help in whatever way I can.

Therapist Agent: 

...

I've connected you with our crisis handler agent who is equipped to help you with these feelings. Please know that you're not alone and there are people who care about you and want to help. Take care of yourself, and reach out whenever you need to.

Therapist Agent: 

This is a big question, and it's okay to take some time to figure it out. Everyone's path is different, and sometimes it takes exploring a few different options to discover what's fulfilling for you.

Are there things you enjoy doing or subjects you enjoy learning about? We could start by looking at those interests and see how they might lead to positive steps forward. If your focus is leaning towards your future or academic goals, I can connect you with our counselor agent for some guidance. Would you like to explore that path?

Therapist Agent: 

That's okay, we can take things one step at a time. If you're feeling a bit unsure, we can just chat about your current interests or anything you feel curious about. There's no rush in figuring things out.

Would you like to share what subjects or hobbies you currently enjoy? Sometimes they can lead to insights about future steps.